# Transformer & BERT, GPT

## 학습목표

- **Transformer의 장점과 주요 프로세스인 Self-Attention에 대해 이해하고 설명할 수 있다.** 
    - 트랜스포머를 발표한 논문 제목은 왜 "Attention is All You Need"인지 설명할 수 있다.
    - Positional Encoding을 적용하는 이유에 대해서 설명할 수 있다.
    - Masked Self-Attention가 트랜스포머 구조 중 어디에 적용되며 어떤 역할을 하는지 설명할 수 있다. 
    - 기존 RNN과 비교하여 Transformer가 가지는 장점에 대해서 설명할 수 있다.

- **GPT, BERT 그리고 다른 모델에 대해서 개략적으로 설명할 수 있다.**
    - GPT(Generative Pre-Training)
        - 사전 학습된 언어 모델(Pre-trained LM)의 Pre-training과 Fine-tuning은 무엇이고 각각 어떤 종류의 데이터셋을 사용하는 지 설명할 수 있다.
        - GPT는 Transformer를 어떻게 변형하였는지 설명할 수 있다.
    - BERT(Bidirectional Encoder Representation by Transformer)
        - BERT는 Transformer를 어떻게 변형하였으며 GPT와의 차이 무엇인지 알 수 있다.
        - MLM(Masked Language Model)은 무엇인지 이해할 수 있다.
        - NSP(Next Sentence Prediction)은 무엇인지 이해할 수 있다.
- **최근 언어 모델의 발전은 어떻게 진행되고 있는지 알 수 있다.**

- 주의 : 이후에도 새로운 모델을 공부하게 되면 아래와 같은 순서로 학습하게 될 겁니다.
    1. 논문을 통해 해당 모델의 컨셉과 대략적인 구조를 파악한다.
    2. 기존에 구현되어 있는 라이브러리를 통해 해당 모델을 사용해본다. (추론 혹은 파인튜닝)
    3. 해당 모델에 대한 좀 더 깊은 이해가 필요할 경우 코드로 구현해본다.
    - 즉, 오늘은 첫 번째로 아래 개념을 배우는 만큼 모든 코드를 이해하지 못해도 괜찮습니다.


## 1. Transformer : Attention is All You Need

### 트랜스포머(Transformer)란?

**<font color="ff6f61">트랜스포머(Transformer)</font>**는 기계 번역을 위한 새로운 모델로 이전에 등장했던 **Attention 메커니즘을 극대화**하여 뛰어난 번역 성능을 기록했습니다.<br/>
최근 자연어처리 모델 SOTA(State-of-the-Art)의 기본 아이디어는 거의 모두 트랜스포머를 기반으로 하고 있습니다.<br/>
모델을 소개한 논문 [Attention is All You Need](https://arxiv.org/abs/1706.03762) 는 3년 사이에 18000번 이상 인용되었습니다.<br/>
트랜스포머가 자연어처리가 아닌 다른 문제도 잘 풀고있기 때문에 최근에는 컴퓨터 비전 쪽에서도 적용하려는 시도가 있으며, 멀티모달(Multi-Modal) 모델에도 적용되고 있습니다.<br/>

RNN 기반 모델이 가진 구조적 단점은 단어가 **순서대로** 들어온다는 점입니다.<br/>
그렇기 때문에 처리해야 하는 시퀀스가 길수록 **연산 시간이** 길어집니다.<br/>
**트랜스포머는 이런 문제를 해결하기 위해 등장한 모델**입니다.<br/>
모든 토큰을 동시에 입력받아 병렬 연산하기 때문에 GPU 연산에 최적화 되어 있습니다.

아래는 트랜스포머의 구조를 단순하게 시각화한 그림입니다.<br/>
Encoder, Decoder로 표현된 사각형을 각각 인코더 블록과 디코더 블록이라고 합니다.<br/>
트랜스포머는 **인코더 블록과 디코더 블록이 6개씩** 모여있는 구조를 하고 있습니다. 

<img src="http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png" alt="positional_encoding" width="700" />

그림 하나를 더 보도록 하겠습니다. 아래는 논문에서 트랜스포머의 구조를 나타낸 그림입니다.<br/>
그림을 보면 커다란 회색 블록이 2개 있습니다.<br/>
왼쪽은 인코더 블록 하나를 나타내고 오른쪽은 디코더 블록 하나를 나타냅니다.<br/>
인코더 블록은 크게 **2개의 sub-layer ▶️ [`Multi-Head (Self) Attention`, `Feed Forward`]** 로 나눌 수 있습니다.<br/>
디코더 블록은 **3개의 sub-layer ▶️ [`Masked Multi-Head (Self) Attention`, `Multi-Head (Encoder-Decoder) Attention`, `Feed Forward`]** 로 나눌 수 있습니다.



<img src="https://miro.medium.com/max/1400/1*BHzGVskWGS_3jEcYYi6miQ.png" alt="positional_encoding" width="550" />

트랜스포머에서는 병렬화를 위해 모든 단어 벡터를 동시에 입력받습니다.<br/>
컴퓨터는 어떤 단어가 어디에 위치하는지 알 수 없게 됩니다.<br/>
그래서 컴퓨터가 이해할 수 있도록 단어의 위치 정보를 제공하기 위한 벡터를 따로 제공해주어야 합니다.<br/>
단어의 상대적인 위치 정보를 제공하기 위한 벡터를 만드는 과정을 **<font color="ff6f61">Positional Encoding</font>** 이라고 합니다.<br/>

Positional Encoding 은 아래와 같은 수식으로 이루어집니다.<br/>


$$
\begin{aligned}
\text{PE}_{\text{pos},2i} &= \sin \bigg(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\bigg) \\
\text{PE}_{\text{pos},2i+1} &= \cos \bigg(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\bigg)
\end{aligned}
$$


아래는 Positional Encoding을 통해 만들어진 벡터를 시각화한 자료입니다.<br/>
일정한 패턴이 있는 벡터가 만들어지는 것을 볼 수 있습니다.<br/>
컴퓨터는 이를 통해 단어의 상대적인 위치를 파악하게 됩니다.<br/>

<img src="http://jalammar.github.io/images/t/transformer_positional_encoding_large_example.png" alt="positional_encoding" width="500" />

In [1]:
def get_angles(pos, i, d_model):
    """
    sin, cos 안에 들어갈 수치를 구하는 함수입니다.
    """
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

In [2]:
def positional_encoding(position, d_model):
    """
    위치 인코딩(Positional Encoding)을 구하는 함수입니다.
    
    """
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

### Self-Attention (셀프-어텐션)





<img width="300" alt="self-Attn" src="https://user-images.githubusercontent.com/45377884/112809266-ca735080-90b4-11eb-9a25-7f34f37880c7.png">

트랜스포머의 주요 메커니즘인 **<font color="ff6f61">Self-Attention</font>** 에 대해 알아보도록 하겠습니다.

아래와 같은 문장이 있다고 해보겠습니다.

> *The animal didn't cross the street because <ins>it</ins> was too tired* 

위 문장을 제대로 번역하려면 **_"it"_** 과 같은 지시대명사가 어떤 대상을 가리키는지 알아야 합니다.<br/>
그렇기 때문에 트랜스포머에서는 번역하려는 문장 내부 요소의 관계를 잘 파악하기 위해서, 문장 자신에 대해 어텐션 메커니즘을 적용합니다.<br/>
이를 **Self-Attention** 이라고 합니다.

아래는 **_"it"_** 이 어떤 단어와 가장 연관되어 있는 지를 시각화한 그림입니다.

<img src="http://jalammar.github.io/images/t/transformer_self-attention_visualization.png" alt="self_attention_visualization" width="350" />

**Self-Attention**은 어떤 과정이길래 단어 사이의 관계를 알아낼 수 있을까요?

Self-Attention에서도 쿼리(Query)-키(Key)-밸류(Value)의 아이디어가 동일하게 등장합니다.


- **지난 시간에 배운 내용을 복습해봅시다 - 검색 시스템에서 흘러나온 쿼리, 키, 밸류**

Attention을 다룰 때 등장했던 검색 시스템의 아이디어를 다시 복습해봅시다.

아래는 구글에서 _"what is attention in nlp"_ 라는 검색어를 구글에 입력했을 때의 검색 결과를 나타낸 이미지입니다.

<img src="https://i.imgur.com/JdCQr1l.png" alt="retrieval_system" width="600" />

그림에서 볼 수 있듯이 검색 시스템은 아래와 같은 3단계를 거쳐 작동합니다.

1. 찾고자 하는 정보에 대한 검색어(Query)를 입력합니다.
2. 검색 엔진은 검색어와 가장 비슷한 키워드(Key)를 찾습니다.
3. 그리고 해당 키워드(Key)와 연결된 페이지(Value)를 보여줍니다.

**기존 Attention과의 차이는 각 벡터가 모두 가중치 벡터라는 점**입니다.<br/>

각각의 벡터가 어떤 역할을 하는지 알아보겠습니다.

- **쿼리(q)**는 분석하고자 하는 단어에 대한 가중치 벡터입니다.

- **키(k)**는 각 단어가 쿼리에 해당하는 단어와 얼마나 연관있는 지를 비교하기 위한 가중치 벡터입니다.

- **밸류(v)**는 각 단어의 의미를 살려주기 위한 가중치 벡터입니다.

**Self-Attention**은 세 가지 가중치 벡터를 대상으로 어텐션을 적용합니다.<br/>
적용하는 방식은 기존 Attention 메커니즘과 거의 동일합니다.

1. 먼저, **특정 단어의 쿼리(q) 벡터와 모든 단어의 키(k) 벡터를 내적**합니다. 내적을 통해 나오는 값이 Attention 스코어(Score)가 됩니다.

2. 트랜스포머에서는 이 가중치를 q,k,v 벡터 차원 $d_k$ 의 제곱근인 $\sqrt{d_k}$ 로 나누어줍니다.<br/>계산값을 안정적으로 만들어주기 위한 계산 보정으로 생각해주시면 됩니다.  

3. 다음으로 **Softmax**를 취해줍니다.<br/>
이를 통해 쿼리에 해당하는 단어와 문장 내 다른 단어가 가지는 관계의 비율을 구할 수 있습니다.

4. 마지막으로 **밸류(v) 각 단어의 벡터를 곱해준 후 모두 더하면** Self-Attention 과정이 마무리됩니다. 

**Self-Attention** 의 과정을 그림으로 다시 보겠습니다.

**1. 가중치 행렬 $W^Q, W^K, W^V$ 로부터 각 단어의 쿼리, 키, 밸류(q, k, v) 벡터를 만들어냅니다.**

<img src="http://jalammar.github.io/images/xlnet/self-attention-1.png" alt="transformer_15" width="600" />

**2. 분석하고자 하는 단어의 쿼리 벡터(q)와 문장 내 모든 단어(자신 포함)의 키 벡터(k)를 내적하여 각 단어와 얼마나 관련 정도를 구합니다.**

(아래 그림에서는 $\sqrt{d_k}$로 나누어 준 뒤에 Softmax를 취해주는 과정은 생략되었습니다.)


<img src="http://jalammar.github.io/images/xlnet/self-attention-2.png" alt="transformer_15" width="600" />

**3.  Softmax의 출력값과밸류 벡터(v)를 곱해준 뒤 더하면 해당 단어에 대한 Self-Attention 출력값을 얻을 수 있습니다.**

<img src="http://jalammar.github.io/images/xlnet/self-attention-3.png" alt="transformer_15" width="600" />

실제로 각 벡터는 **행렬(Q, K, V)**로 한꺼번에 계산됩니다. $W^Q, W^K, W^V$ 는 학습 과정에서 갱신되는 파라미터로 이루어진 행렬입니다.<br/>
세 행렬과 단어 행렬을 내적하여 쿼리, 키, 밸류 행렬(Q, K, V)를 만들어냅니다.



<img src="http://jalammar.github.io/images/t/self-attention-matrix-calculation.png" alt="transformer_12" width="400" />

위에서 살펴본 바와 같이

1. 먼저 쿼리 행렬(Q)과 키 행렬(K)을 **내적**합니다.

2. 결과로 나오는 행렬의 요소를 $\sqrt{d_k}$ 로 **나누어 줍니다.**

3. 행렬의 각 요소에 **소프트맥스(Softmax)**를 취해줍니다. 

4. 마지막으로 **밸류 행렬(V)과 내적**하면 최종 결과 행렬(Z)이 반환됩니다.

<img src="http://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png" alt="transformer_13" width="700" />

아래는 Tensorflow 에서 Self-Attention을 구현한 코드입니다. 코드를 통해 Attention이 계산되는 과정을 다시 살펴보도록 합시다.

In [3]:
def scaled_dot_product_attention(q, k, v, mask):
    """
    Attention 가중치를 구하는 함수입니다.
    q, k, v 의 leading dimension은 동일해야 합니다.
    k, v의 penultimate dimension이 동일해야 합니다, i.e.: seq_len_k = seq_len_v.

    Mask는 타입(padding or look ahead)에 따라 다른 차원을 가질 수 있습니다.
    덧셈시에는 브로드캐스팅 될 수 있어야합니다.
    
    Args:
        q: query shape == (..., seq_len_q, depth)
        k: key shape == (..., seq_len_k, depth)
        v: value shape == (..., seq_len_v, depth_v)
        mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.
        
    Returns:
        output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    
    # matmul_qk(쿼리와 키의 내적)을 dk의 제곱근으로 scaling 합니다.
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # 마스킹을 진행합니다.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # 소프트맥스(softmax) 함수를 통해서 attention weight 를 구해봅시다.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

### Multi-Head Attention

다음으로 **<font color="ff6f61">Multi-Head Attention</font>** 에 대해 알아보겠습니다.<br/>
Multi-Head Attention 이란 **Self-Attention을 동시에 병렬적으로 실행하는 것**입니다.<br/>
각 Head 마다 다른 Attention 결과를 내어주기 때문에 앙상블과 유사한 효과를 얻을 수 있습니다.<br/> 
논문에서는 8개의 Head를 사용하였습니다.<br/>
8번의 Self-Attention을 실행하여 각각의 출력 행렬 $Z_0, Z_1, \cdots , Z_7$ 을 만들어냅니다.

<img src="http://jalammar.github.io/images/t/transformer_attention_heads_z.png" alt="transformer_16" width="500"/>

출력된 행렬 $Z_n (n=0,\cdots,7)$ 은 **이어붙여집니다(Concatenate)**.<br/>
또 다른 파라미터 행렬인 $W^o$ 와의 내적을 통해 Multi-Head Attention의 최종 결과인 행렬 $Z$를 만들어냅니다.<br/>
여기서 행렬 $W^o$의 요소 역시 학습을 통해 갱신됩니다.<br/>
최종적으로 생성된 행렬 $Z$는 토큰 벡터로 이루어진 행렬 $X$와 **동일한 크기(Shape)**가 됩니다.

<img src="http://jalammar.github.io/images/t/transformer_attention_heads_weight_matrix_o.png" alt="transformer_17" width="500" />

### Layer Normalization & Skip Connection



<img width="300" alt="lnorm_resicon" src="https://user-images.githubusercontent.com/45377884/113169444-9056aa00-9280-11eb-8ba0-17c9211ad412.png">

트랜스포머의 모든 sub-layer에서 출력된 벡터는 **Layer normalization**과 **Skip connection**을 거치게 됩니다.<br/>
Layer normalization의 효과는 Batch normalization과 유사합니다. 학습이 훨씬 빠르고 잘 되도록 합니다.<br/>
Skip connection(혹은 Residual connection)은 역전파 과정에서 정보가 소실되지 않도록 합니다.<br/>

### Feed Forward Neural Network

<img width="300" alt="스크린샷 2021-03-29 오후 5 27 32" src="https://user-images.githubusercontent.com/45377884/112808809-58027080-90b4-11eb-8ca7-ffa38e577d3d.png">



다음으로 **<font color="ff6f61">FFNN(Feed forward neural network)</font>** 로 들어갑니다.<br/>은닉층의 차원이 늘어났다가 다시 원래 차원으로 줄어드는 단순한 2층 신경망입니다.<br/>활성화 함수(Activation function)으로 ReLU를 사용합니다.

$$
 \text{FFNN}(x) = \max(0, W_1x + b_1) W_2 +b_2
$$

In [4]:
def point_wise_feed_forward_network(d_model, dff):
    """
    FFNN을 구현한 코드입니다.

    Args:
        d_model : 모델의 차원입니다.
        dff : 은닉층의 차원 수입니다. 논문에서는 2048을 사용하였습니다.
    """
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

### Masked Self-Attention


<img width="300" alt="Masked_Self-Attention_in_structure" src="https://user-images.githubusercontent.com/45377884/112808936-78322f80-90b4-11eb-9315-22cd9caad41d.png">

**<font color="ff6f61">Masked Self-Attention</font>**은 디코더 블록에서 사용되는 특수한 Self-Attention입니다.<br/>
디코더는 Auto-Regressive(왼쪽 단어를 보고 오른쪽 단어를 예측)하게 단어를 생성하기 때문에 타깃 단어 이후 단어를 보지 않고 단어를 예측해야 합니다.<br/>
따라서 타깃 단어 뒤에 위치한 단어는 Self-Attention에 영향을 주지 않도록 **마스킹(masking)**을 해주어야 합니다.

<img width="500" alt="Masked_Self-Attention_ex" src="http://jalammar.github.io/images/xlnet/transformer-decoder-block-self-attention-2.png">

***Self-Attention (without Masking) vs Masked Self-Attention***

<img width="500" alt="Masked_Self-Attention_ex2" src="http://jalammar.github.io/images/gpt2/self-attention-and-masked-self-attention.png">

Self-Attention 메커니즘은 쿼리 행렬(Q)와 키 행렬(K)의 내적합니다.<br/>
결과로 나온 행렬을 차원의 제곱근 $\sqrt{d_k}$ 로 나누어 준 다음,<br/> 
Softmax를 취해주고 밸류 행렬(V)과 내적하였습니다.

**Masked Self-Attention** 에서는 Softmax를 취해주기 전, 가려주고자 하는 요소에만 $-\infty$ 에 해당하는 매우 작은 수를 더해줍니다.<br/>
아래 코드 예시에서는 -10억(=`-1e9`)을 더해주었습니다.<br/>
이 과정을 **마스킹(Masking)**이라고 합니다.<br/>
마스킹된 값은 Softmax를 취해 주었을 때 0이 나오므로 Value 계산에 반영되지 않습니다.

<img width="600" alt="masked_1" src="http://jalammar.github.io/images/gpt2/transformer-attention-mask.png">

<img width="600" alt="masked_2" src="http://jalammar.github.io/images/gpt2/transformer-attention-masked-scores-softmax.png">

위에서 등장했던 Self-Attention을 구현 코드에서 `mask` 와 관련된 부분만 다시 보도록 합시다.

In [5]:
def scaled_dot_product_attention(q, k, v, mask):
    """
    Attention 가중치를 구하는 함수입니다.
    q, k, v 의 leading dimension은 동일해야 합니다.
    k, v의 penultimate dimension이 동일해야 합니다, i.e.: seq_len_k = seq_len_v.

    Mask는 타입(padding or look ahead)에 따라 다른 차원을 가질 수 있습니다.
    덧셈시에는 브로드캐스팅 될 수 있어야합니다.
    
    Args:
        q: query shape == (..., seq_len_q, depth)
        k: key shape == (..., seq_len_k, depth)
        v: value shape == (..., seq_len_v, depth_v)
        mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.
        
    Returns:
        output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    
    # matmul_qk(쿼리와 키의 내적)을 dk의 제곱근으로 scaling 합니다.
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    """
    mask가 있을 경우 masking된 자리(mask=1)에는 (-inf)에 해당하는 절댓값이 큰 음수 -1e9(=-10억)을 더해줍니다.
    그 값에 softmax를 취해주면 거의 0에 가까운 값이 나옵니다. 그 다음 value 계산시에 반영되지 않습니다.
    """

    # 마스킹을 진행합니다.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # 소프트맥스(softmax) 함수를 통해서 attention weight 를 구해봅시다.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

### Encoder-Decoder Attention

<img width="300" alt="Encoder-Decoder_Attention" src="https://user-images.githubusercontent.com/45377884/112809435-f8f12b80-90b4-11eb-96e1-3b0f7c530659.png">

디코더에서 Masked Self-Attention 층을 지난 벡터는 **<font color="ff6f61">Encoder-Decoder Attention</font>** 층으로 들어갑니다.<br/>
좋은 번역을 위해서는 **번역할 문장과 번역된 문장 간의 관계** 역시 중요합니다.<br/>
번역할 문장과 번역되는 문장의 정보 관계를 엮어주는 부분이 바로 이 부분입니다.

이 층에서는 **디코더 블록의** Masked Self-Attention으로부터 출력된 벡터를 **쿼리(Q)** 벡터로 사용합니다.<br/>
**키(K)와 밸류(V)** 벡터는 최상위(=6번째) 인코더 블록에서 사용했던 값을 그대로 가져와서 사용합니다.<br/>
**Encoder-Decoder Attention** 층의 계산 과정은 Self-Attention 했던 것과 동일합니다.

아래는 **Encoder-Decoder Attention** 가 진행되는 순서를 나타낸 이미지입니다.

<img width="700" alt="Encoder-Decoder_Attention_gif" src="http://jalammar.github.io/images/t/transformer_decoding_1.gif">

### Linear & Softmax Layer


<img width="300" alt="Linear_Softmax" src="https://user-images.githubusercontent.com/45377884/112815762-994a4e80-90bb-11eb-9a57-a8be65c1a30b.png">

디코더의 최상층을 통과한 벡터들은 Linear 층을 지난 후 Softmax를 통해 예측할 단어의 확률을 구하게 됩니다.

## 코드 실습 1

(오래 걸립니다. 원활한 실습을 위해서 epoch 수를 줄이고 학습해도 좋습니다.)

필요한 모듈을 import 해줍니다.

In [ ]:
#!pip install tensorflow==2.12.0

In [38]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

학습팔 말뭉치를 불러옵니다.<br/>
아래 코드에서는 스페인어 - 영어 말뭉치를 불러옵니다.

In [39]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

학습할 수 있도록 적절히 전처리를 해줍니다.

In [40]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

전처리가 잘 되었는지 확인합니다. <br/>
번역되는 문장의 앞에는 [start] 토큰을 위치시키고 뒤에는 [end] 토큰을 위치시킵니다.

In [41]:
for _ in range(5):
    print(random.choice(text_pairs))

('He is as poor as can be.', '[start] Él es pobre como pocos. [end]')
('It is mine.', '[start] Es mío. [end]')
('The extremists refused to negotiate.', '[start] Los extremistas rechazaron negociar. [end]')
('I like to dance.', '[start] Me gusta bailar. [end]')
('I have a meeting this afternoon.', '[start] Tengo una reunión esta tarde. [end]')


데이터셋을 split 해줍니다.

In [42]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


In [43]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

In [44]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return ({"encoder_inputs": eng, "decoder_inputs": spa[:, :-1],}, spa[:, 1:])


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [45]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [46]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [47]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [48]:
epochs = 1  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_4 (Positi  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder_2 (Transfo  (None, None, 256)   3155456     ['positional_embedding_

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ('self',)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ('self',)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1302/1302 [==============================] - ETA: 0s - loss: 3.8194 - accuracy: 0.4366

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1302/1302 [==============================] - 117s 83ms/step - loss: 3.8194 - accuracy: 0.4366 - val_loss: 2.8750 - val_accuracy: 0.5379


In [49]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)

## 코드 실습 2

In [50]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [51]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [52]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [53]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

17464789/17464789 [==============================] - 0s 0us/step
25000 Training sequences
25000 Validation sequences


In [54]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [55]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val)
)

Epoch 1/2


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ('self',)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ('self',)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
782/782 [==============================] - ETA: 0s - loss: 0.3895 - accuracy: 0.8113

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
782/782 [==============================] - 70s 82ms/step - loss: 0.3895 - accuracy: 0.8113 - val_loss: 0.3246 - val_accuracy: 0.8668
Epoch 2/2
782/782 [==============================] - 21s 26ms/step - loss: 0.1987 - accuracy: 0.9241 - val_loss: 0.3072 - val_accuracy: 0.8699


## Review

- Attention의 장점에 대해서 생각하고 설명해봅니다.

    - RNN 모델의 단점 2가지
    - 장기 의존성(Long-term dependency)
    - Attention의 장점


- GPT & BERT
    - 사전 학습 언어 모델(Pretrained Language Model), 전이 학습(Transfer Learning)
        - 사전 학습(Pre-training)
        - Fine-tuning
    - GPT의 구조
    - BERT의 구조
        - MLM(Masked Langauge Model)
        - NSP(Next Sentence Prediction)

        

## References

- 트랜스포머에 대해 조금 더 자세하게 알고 싶다면
    - [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
    - [번역](https://nlpinkorean.github.io/illustrated-transformer/)
    - [Paper](https://arxiv.org/pdf/1706.03762.pdf) (Attention is All You Need)

- GPT에 대해 더 자세하게 알고 싶다면
    - [The Illustrated GPT-2](http://jalammar.github.io/illustrated-gpt2/) (Visualizing Transformer Language Models)
    - [Paper](https://www.cs.ubc.ca/~amuham01/LING530/papers/radford2018improving.pdf) (Improving Language Understanding by Generative Pre-Training)

- BERT에 대해 더 자세하게 알고 싶다면
    - [The Illustrated BERT, ELMo, and co.](http://jalammar.github.io/illustrated-bert/) (How NLP Cracked Transfer Learning)
    - [번역](https://nlpinkorean.github.io/illustrated-bert/)
    - [Paper](https://arxiv.org/pdf/1810.04805.pdf) (Pre-training of Deep Bidirectional Transformers for
Language Understanding)